![Profiling python code with cProfile](images/cProfile_20220809.png "Profiling python code with cProfile")

# Background

A [software stack for streamflow forecasting](https://github.com/csiro-hydroinformatics/streamflow-forecasting-tools-onboard/) consists of a C++ modelling engine, accessible via a C API from various interactive language bindings including python. I have not done some benchmark performance profiling for a while on this stack. I expect the C++ core to be pretty much as good as it was 8 years ago, but as I bed down some of the python bindings, I notice some lags when interacting. This is not a surprise, and I expect these to be either in python code, or perhaps at the boundary with the C API. This intuition needs to be confirmed by objective measurements. 

It is one of these things that are rarely a blocker for quite a while, but a bit of instrumentation and performance tuning improves the user experience, a little bit every day. Also, some use cases where substantial stochastic data is generated in Python and passed to the C++ core would be very penalised by inefficiencies in Python or Python/C interoperability code.

**Foreword**: in the case below, we do not need to profile C++ code _per se_ after all, so if this is what you are after specifically. Read on for the Python side of the story.

# Python profilers

[9 fine libraries for profiling python code](https://www.infoworld.com/article/3600993/9-fine-libraries-for-profiling-python-code.html) is a recent article, as I write. Of these, Palanteer is interesting in its mixed mode profiling capabilities (Python and C++). I'll have to explore it, perhaps not just yet though. A priori the `cProfiler` coming with the Python base library is all I need for this immediate use case. 

# Runtime profiling

I will skip on the details of the imported libraries here.


In [1]:
import pandas as pd
import numpy as np
import xarray as xr
from swift2.simulation import create_subarea_simulation, create_catchment
# etc. etc, other imports

In [2]:
#| include: false
from swift2.utils import mk_full_data_id
from swift2.parameteriser import create_parameteriser, create_parameter_sampler, create_sce_termination_wila, get_default_sce_parameters
from swift2.doc_helper import get_free_params, sample_series

from typing import List

from typing import TYPE_CHECKING
if TYPE_CHECKING:
    from swift2.classes import Simulation

from cinterop.cffi.marshal import TIME_DIMNAME, slice_xr_time_series, pd_series_to_xr_series, as_timestamp

from swift2.system import runoff_model_ids, runoff_model_var_ids
from swift2.utils import paste_2, vpaste

import matplotlib.pyplot as plt

## Creating synthethic hydrologic model

The overall purpose of the exercise will be to measure performance under various conditions: model structure, size, time steps, etc. We will not do all that in this post; suffice to say we define a set of functions to create synthetic model setups. We do not show the full definitions. To give a flavour

In [3]:
from cinterop.timeseries import mk_hourly_xarray_series

def create_pulses(nTimeSteps, start) : return mk_hourly_xarray_series( createPulse(nTimeSteps, 5.0, 48), start)

def create_uniform(nTimeSteps, start) : return mk_hourly_xarray_series( createUniform(nTimeSteps, 1.0), start)

def set_test_input(ms:'Simulation', subAreaName:'str', rainVarName, petVarName, rain_ts, pet_ts):
    p_id = mk_full_data_id('subarea', subAreaName, rainVarName)
    e_id = mk_full_data_id('subarea', subAreaName, petVarName)
    ms.play_input( rain_ts, p_id)
    ms.play_input( pet_ts, e_id)
    
# def create_line_system(n_time_steps, n_links, rain_varname = "P", pet_varname = "E", area_km2 = 1):
# et caetera    

In [4]:
#| include: false
def create_line_system(n_time_steps, n_links, rain_varname = "P", pet_varname = "E", area_km2 = 1):
    node_ids = [str(x) for x in range(1,n_links+2)]
    node_names = ["node_" + str(x) for x in node_ids]
    link_ids = [str(x) for x in range(1,n_links+1)]
    link_names = ['lnk_' + str(x) for x in link_ids]
    link_from_node = node_ids[:-1]
    link_to_node = node_ids[1:] 

    ms = create_catchment(node_ids, node_names, link_ids, link_names, link_from_node, link_to_node)

    sa_ids = ms.get_subarea_ids()

    setTestTimeSpan(n_time_steps, ms)
    start = ms.get_simulation_span()['start']
    rain_ts = create_pulses(n_time_step, start)
    pet_ts = create_uniform(n_time_step, start)

    for sa in sa_ids:
        set_test_input(ms, sa, rain_varname, pet_varname, rain_ts, pet_ts)
    return ms

def setTestTimeSpan(nTimeSteps, ms:'Simulation'):
    startDate = pd.Timestamp(year=1989, month=1, day=1)
    endDate = startDate + pd.DateOffset(hours=(nTimeSteps-1))
    ms.set_simulation_span(startDate, endDate)
    ms.set_simulation_time_step("hourly")


def _moduval(n, value):
    def f(i):
        return value if i % n == 0 else 0
    return f

def createPulse(nTimeSteps, value, pulsePeriod):
    f = _moduval(pulsePeriod, value)
    x = [f(i) for i in range(nTimeSteps)]
    return np.array(x).astype(float)

def createUniform(nTimeSteps, value):
    x = np.empty(shape=(nTimeSteps,))
    x[:] = value
    return x.astype(float)

We are now ready to create our catchment simulation. Before we plunge into `cProfiler` let's use a simpler way to assess the runtime from notebooks:

## Using notebook's `%%time`

Let's create a setup with 15 subareas, hourly time step over 10 years.

In [5]:
%%time 
n_time_step = 10 * 365 * 24
ms = create_line_system(n_time_step, 15)

CPU times: user 888 ms, sys: 8.94 ms, total: 897 ms
Wall time: 897 ms


Well, this was only the create of the baseline model, not even execution, and this already takes close to a second. Granted, there are a fair few hours in a decade. Still, a whole second!

What about the simulation runtime? Let's parameterise minimally to avoid possible artefacts, and execute.

In [6]:
from swift2.doc_helper import configure_hourly_gr4j, get_free_params
configure_hourly_gr4j(ms)
p = create_parameteriser('Generic subarea', get_free_params("GR4J"))
p.apply_sys_config(ms)

Double check we are indeed running hourly over 10 years:

In [7]:
ms.get_simulation_span()

{'start': datetime.datetime(1989, 1, 1, 0, 0),
 'end': datetime.datetime(1998, 12, 29, 23, 0),
 'time step': 'hourly'}

In [8]:
%%time
ms.exec_simulation()

CPU times: user 314 ms, sys: 691 µs, total: 315 ms
Wall time: 314 ms


This is actually quite good, and "unexpectedly" less than the model creation itself. This is actually not all that surprising. All of the model execution happens in C++ land. The model setup involves much more operations in python.

Let's look at an operation exchanging data from the C++ engine for display in Python. The model simulation has some of its states receiving input time series:  

In [9]:
ms.get_played_varnames()[:6]

['subarea.1.E',
 'subarea.1.P',
 'subarea.10.E',
 'subarea.10.P',
 'subarea.11.E',
 'subarea.11.P']

Let's see what happens in the retrieval of one of these input time series:

In [10]:
%%time
ts = ms.get_played("subarea.1.E")

CPU times: user 441 ms, sys: 106 µs, total: 441 ms
Wall time: 440 ms


This is substantial; more than the native execution over a catchment with 15 subareas. So:

* Can we identify the hotspot(s)?
* Can we do something to improve it.

## Profiling

Enter `cProfile`, as we will stick with this in this post. Adapting some of the sample code shown in the [Python documentation on profilers](https://docs.python.org/3/library/profile.html)

In [11]:
import cProfile

In [12]:
import pstats, io
pr = cProfile.Profile()
pr.enable()
ts = ms.get_played("subarea.1.E")
pr.disable()

In [13]:
s = io.StringIO()
sortby = pstats.SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)

We will print only the top 5 % of the list of function calls, and see if we can spot the likely hotspot

In [14]:
ps.print_stats(.05)
print(s.getvalue())

         6137 function calls (5964 primitive calls) in 0.445 seconds

   Ordered by: cumulative time
   List reduced from 588 to 29 due to restriction <0.05>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.445    0.222 /home/per202/miniconda/envs/hydrofc/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361(run_code)
        2    0.000    0.000    0.445    0.222 {built-in method builtins.exec}
        1    0.000    0.000    0.445    0.445 /tmp/ipykernel_34182/2688031072.py:4(<cell line: 4>)
        1    0.000    0.000    0.445    0.445 /home/per202/src/swift/bindings/python/swift2/swift2/classes.py:471(get_played)
        1    0.000    0.000    0.445    0.445 /home/per202/src/swift/bindings/python/swift2/swift2/play_record.py:190(get_played)
        1    0.000    0.000    0.444    0.444 /home/per202/src/swift/bindings/python/swift2/swift2/internal.py:94(internal_get_played_tts)
        1    0.000    0.000    0.444   

in a file rcpp-interop-commons/bindings/python/cinterop/cinterop/timeseries.py the function `create_even_time_index` appears to be where a lengthy operation occurs. More precisely, when this function does a list comprehension (`listcomp`). Note that I infer this because in the `cumtime` (cumulative time) column there is a drop from 0.396 ms for `listcomp` to 0.071 for the rest of the operations under this function. I think this is the right way to interpret it in this case, but it may not be the case in other profiling context.

The code for `create_even_time_index` is at [this permalink](https://github.com/csiro-hydroinformatics/rcpp-interop-commons/blob/f7dff4b83d8f01a8ae71e16fb903b80fa4f23f5d/bindings/python/cinterop/cinterop/timeseries.py#L22)

```python
def create_even_time_index(start:ConvertibleToTimestamp, time_step_seconds:int, n:int) -> List:
    start = as_timestamp(start)
    delta_t = np.timedelta64(time_step_seconds, 's')
    return [start + delta_t * i for i in range(n)]
```

`[start + delta_t * i for i in range(n)]` is the bulk of the time, .396 out of a total 0.477 ms. 

This list is created as the basis for a time index for the creation of the `xarray` object returned by the overall `get_played` function. So, is there a faster way to create this time series index?

## Performance tuning


In [15]:
start = pd.Timestamp(year=2000, month=1, day=1)

In [16]:
n= 24*365*10

In [17]:
def test_index_creation(start, n:int) -> List:
    start = as_timestamp(start)
    time_step_seconds = 3600
    delta_t = np.timedelta64(time_step_seconds, 's')
    return [start + delta_t * i for i in range(n)]

In [18]:
%%time
a = test_index_creation(start, n)

CPU times: user 352 ms, sys: 562 µs, total: 353 ms
Wall time: 351 ms


`start` is a pandas `Timestamp`, and we add to it an object of type `np.timedelta64` 87600 times. I doubt this is the main issue, but let's operate in numpy types as much as we can by converting the pd.Timestamp once:

In [19]:
start

Timestamp('2000-01-01 00:00:00')

In [20]:
start.to_datetime64()

numpy.datetime64('2000-01-01T00:00:00.000000000')

In [21]:
def test_index_creation(start, n:int) -> List:
    start = as_timestamp(start).to_datetime64()
    time_step_seconds = 3600
    delta_t = np.timedelta64(time_step_seconds, 's')
    return [start + delta_t * i for i in range(n)]

In [22]:
%%time
a = test_index_creation(start, n)

CPU times: user 293 ms, sys: 8.48 ms, total: 301 ms
Wall time: 300 ms


This is actually more of an improvement than I anticipated. OK. What else can we do?

Pandas has the helpful [Time series / date functionality](https://pandas.pydata.org/docs/user_guide/timeseries.html) page in its documentation. The function from which we started is generic, but for important cases such as hourly and daily time steps, there are options to use the `freq` argument to the `date_range` function

In [23]:
%%time
pd.date_range(start, periods=n, freq="H")

CPU times: user 242 µs, sys: 686 µs, total: 928 µs
Wall time: 520 µs


DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:00:00',
               '2000-01-01 02:00:00', '2000-01-01 03:00:00',
               '2000-01-01 04:00:00', '2000-01-01 05:00:00',
               '2000-01-01 06:00:00', '2000-01-01 07:00:00',
               '2000-01-01 08:00:00', '2000-01-01 09:00:00',
               ...
               '2009-12-28 14:00:00', '2009-12-28 15:00:00',
               '2009-12-28 16:00:00', '2009-12-28 17:00:00',
               '2009-12-28 18:00:00', '2009-12-28 19:00:00',
               '2009-12-28 20:00:00', '2009-12-28 21:00:00',
               '2009-12-28 22:00:00', '2009-12-28 23:00:00'],
              dtype='datetime64[ns]', length=87600, freq='H')

It is two order of magnitude faster... Definitely worth a re-engineering of the features in the `timeseries.py` we started from. 

This probably does not solve the issue for many other cases (irregular time steps, e.g. monthly), but there are many cases where we could benefit. The [date_range documentation](https://pandas.pydata.org/docs/reference/api/pandas.date_range.html?highlight=date_range) specifies that an arbitrary [DateOffset](https://pandas.pydata.org/docs/reference/api/pandas.tseries.offsets.DateOffset.html) to its `freq` argument (`freq: str or DateOffset, default ‘D’`). How efficient is this operation on our 87600 data points?

In [24]:
d_offset = pd.tseries.offsets.DateOffset(minutes=15)

In [25]:
start + d_offset

Timestamp('2000-01-01 00:15:00')

In [26]:
%%time
pd.date_range(start, periods=n, freq=d_offset)

CPU times: user 1.5 s, sys: 1.32 ms, total: 1.5 s
Wall time: 1.5 s


DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 00:15:00',
               '2000-01-01 00:30:00', '2000-01-01 00:45:00',
               '2000-01-01 01:00:00', '2000-01-01 01:15:00',
               '2000-01-01 01:30:00', '2000-01-01 01:45:00',
               '2000-01-01 02:00:00', '2000-01-01 02:15:00',
               ...
               '2002-07-01 09:30:00', '2002-07-01 09:45:00',
               '2002-07-01 10:00:00', '2002-07-01 10:15:00',
               '2002-07-01 10:30:00', '2002-07-01 10:45:00',
               '2002-07-01 11:00:00', '2002-07-01 11:15:00',
               '2002-07-01 11:30:00', '2002-07-01 11:45:00'],
              dtype='datetime64[ns]', length=87600, freq='<DateOffset: minutes=15>')

It looks like in this case this is actually a fair bit slower than my original implementation. Interesting. And using `start.to_datetime64()` makes no difference too.  

# Conclusion

This demonstrated a relatively simple, but real case where `cProfiler` helps alleviate usually small but pervasive runtime inefficiencies. In this case, so far we have not needed to look close to the Python/C interoperability layer. The key bottleneck was pure python. I envisage I may post something later on looking at trickier situation.

To be honest, I did second guess a few things upfront. But the `cProfiler` and simpler `%%time` brought at least confirmation and sometimes useful, conter-intuitive insight. "You cannot manage what you cannot measure" as the saying goes.
